In [3]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score as auc, accuracy_score
import numpy as np

In [4]:
n = 100
df = pd.read_csv('../prepare_data/data/base_line.csv')
df.drop('match_id', axis=1, inplace=True)
df.head()

,gameweek_gameweek,gameweek_compSeason_label,teams_team_1_name,teams_team_2_name,ground_name,team_1_hue
0,20,2023,West Ham United,Brighton & Hove Albion,London Stadium,2
1,20,2023,Liverpool,Newcastle United,Anfield,1
2,20,2023,Fulham,Arsenal,Craven Cottage,1
3,20,2023,Tottenham Hotspur,Bournemouth,Tottenham Hotspur Stadium,1
4,20,2023,Nottingham Forest,Manchester United,The City Ground,1


In [5]:
train = df.iloc[n:, :]
val = df.iloc[:n, :]

In [6]:
train

,gameweek_gameweek,gameweek_compSeason_label,teams_team_1_name,teams_team_2_name,ground_name,team_1_hue
100,10,2023,Brighton & Hove Albion,Fulham,American Express Stadium,2
101,10,2023,Liverpool,Nottingham Forest,Anfield,1
102,10,2023,West Ham United,Everton,London Stadium,0
103,10,2023,Wolverhampton Wanderers,Newcastle United,Molineux Stadium,2
104,10,2023,Arsenal,Sheffield United,Emirates Stadium,1
...,...,...,...,...,...,...
1713,1,2019,Burnley,Southampton,Turf Moor,1
1714,1,2019,Crystal Palace,Everton,Selhurst Park,2
1715,1,2019,Watford,Brighton & Hove Albion,Vicarage Road,0
1716,1,2019,West Ham United,Manchester City,London Stadium,0


In [7]:
y_train = train['team_1_hue']
x_train = train.drop('team_1_hue', axis=1)

y_val = val['team_1_hue']
x_val = val.drop('team_1_hue', axis=1)

In [8]:
model_baseline = CatBoostClassifier()

In [9]:
model_baseline.fit(x_train, y_train, cat_features=list(x_train.columns), verbose=False)

In [10]:
pred = model_baseline.predict(x_val)
pred_proba = model_baseline.predict_proba(x_val)
y_val = y_val.to_numpy().reshape(-1, 1)

In [11]:
pred = np.concatenate((y_val, pred, pred_proba), axis=1)
df_pred = pd.DataFrame(pred, columns=['true', 'pred', '0_prob', '1_prob', '2_prob'])

In [12]:
df_pred['error'] = np.where(df_pred['true']==df_pred['pred'], 0, 1)
df_pred.head()


,true,pred,0_prob,1_prob,2_prob,error
0,2.0,0.0,0.590681,0.160676,0.248643,1
1,1.0,1.0,0.036982,0.940131,0.022888,0
2,1.0,0.0,0.533615,0.388805,0.077580,1
3,1.0,1.0,0.088459,0.819140,0.092400,0
4,1.0,0.0,0.534983,0.259233,0.205785,1


In [13]:
df_pred['error'].sum()

51

In [14]:
accuracy_score(df_pred['true'], df_pred['pred'])

0.49